# Layerwise learning for Quantum Neural Nets

In [1]:
import random
import collections
import matplotlib.pyplot as plt

# Pennylane
import pennylane as qml
from pennylane import numpy as np

# Pytorch
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

### Parameters

In [2]:
n_qubits = 9
n_quantum_layers = 2
batch_size = 20
epochs = 8

### Data pre-processing

In [3]:
data_transforms = transforms.Compose([transforms.Resize(3), #resize to a 3x3 image
                                      transforms.ToTensor(), #convert to tensor
                                      transforms.Lambda(lambda x: torch.flatten(x)) #obtain a one dimensional vector
                                     ])

In [4]:
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=data_transforms)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=data_transforms)

# Filter to just images of '3's and '6's
subset_indices = ((train_set.targets == 3) + (train_set.targets == 6)).nonzero().view(-1)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False,
                                         sampler=SubsetRandomSampler(subset_indices))

In [5]:
print(subset_indices)

tensor([    7,    10,    12,  ..., 59986, 59996, 59998])


In [6]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([20, 9])
torch.Size([20])


In [7]:
dev = qml.device("default.qubit", wires=n_qubits)

In [8]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

### Variational circuit

We first define some quantum layers that will compose the quantum circuit.

In [9]:
def apply_layer(gates, weights):
    """Docstrings"""
    
    for i in range(n_qubits): 
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])

In [10]:
@qml.qnode(dev, interface="torch")
def quantum_net(inputs, layer_weights):
    """Docstrings"""
        
    wirelist = [i for i in range(n_qubits)]
    
    #Encode the data with Angle Embedding
    qml.templates.AngleEmbedding(inputs, wires=wirelist, rotation='X')
    
    # Sequence of trainable layers
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], layer_weights[i]) #maybe "weights" needs a reformat
        
    # Expectation value of the last qubit
    return qml.expval(qml.PauliZ(n_qubits-1))

In [11]:
# integrate it as a TorchLayer
# https://pennylane.readthedocs.io/en/stable/code/api/pennylane.qnn.TorchLayer.html

In [12]:
# Obtain random gates for each layer
layer_gates = [set_random_gates(n_qubits) for i in range(n_quantum_layers)]

# Define shape of the weights
weight_shapes = {"layer_weights": (n_quantum_layers, n_qubits)}

In [13]:
qlayer = qml.qnn.TorchLayer(quantum_net, weight_shapes, init_method = nn.init.zeros_)
layer_prenet = nn.Linear(n_qubits, n_qubits)
softmax = torch.nn.Softmax(dim=1)

model = torch.nn.Sequential(layer_prenet, qlayer, softmax)

In [14]:
opt = optim.Adam(model.parameters(), lr=0.01)

loss = nn.CrossEntropyLoss()

In [15]:
for epoch in range(epochs):

    for x, y in train_loader:
        opt.zero_grad()

        loss_evaluated = loss(model(x), y)
        loss_evaluated.backward()

        opt.step()


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)